In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [1]:
import pickle
import time
import cknn
import torch
import gc

import numpy as np
import pandas as pd
import DataFormatterEditAddSession as DataFormatterA
import DataFormatterEdit as DataFormatter

from model import *
from tqdm.notebook import tqdm
from utils import build_graph, split_validation, Data

In [3]:
def num_data(data, str_):
    values = data.loc[:,[str_]].values
    values = values.reshape(-1,)
    new = np.unique(values)
    return new.shape[0]

In [4]:
fold = 5
#_path = 'amz_data_prep_2018_book.csv'
_path = 'amz_data_prep_2018_book_40k.csv'

In [5]:
def make_pd_data_frame(train_data, session_key='session_id', item_key='item_id', time_key='eventdate'):
    df = pd.DataFrame({session_key:[], item_key:[], time_key:[]})
    ses_ids = []
    ses_dates = []
    ses_seqs = []
    for ses_id, ses_date, ses_seq in tqdm(zip(train_data[0], train_data[1], train_data[2]), total=len(train_data[0])):
        for ses_ in ses_seq:
            ses_date_ = int(ses_date.astype(np.float))
            ses_ids.append(int(ses_id))
            ses_dates.append(ses_date_)
            ses_seqs.append(ses_)
    ses_ids = np.asarray(ses_ids)
    ses_dates = np.asarray(ses_dates)
    ses_seqs = np.asarray(ses_seqs)
    d = np.concatenate((ses_ids,ses_seqs,ses_dates))
    d = np.reshape(d,(3,-1))
    d = d.T
    column_names = [session_key, item_key, time_key]
    df = pd.DataFrame(d, columns=column_names)
    return df

In [ ]:
data = pd.read_csv(_path, delimiter=',')
n_item = num_data(data, 'item_id')
k_split_data_list, sess_clicks = DataFormatterA.get_k(_path, fold)
last_sr_gnn_hit, sr_gnn_hit, iknn_hit, final_hit, last_sr_gnn_mrr, sr_gnn_mrr, iknn_mrr, final_mrr = [], [], [], [], [], [], [], []

In [ ]:
for __p in range(5):
    avg_p = []
    avg_m = []
    for fold_ in range(fold):
        gc.collect()
        test_data_preproc = k_split_data_list[fold_]
        train_data_preproc = k_split_data_list.copy()
        del train_data_preproc[fold_]
        train_data_ = _ = train_data = test_data = df = knn = mrr_20 = p_20 = scr = scr_convd = slices = None

        train_data_preproc, test_data_preproc, train_data_, _ = DataFormatterA.make_tr_ts_data_set(train_data_preproc, 
                                                                                                      test_data_preproc, 
                                                                                                      sess_clicks)

        train_data = Data(train_data_preproc, shuffle=True)
        test_data = Data(test_data_preproc, shuffle=False)
        #test_data.inputs.shape
        #train_data_preproc = [[subset],[ans_items]]
        #train_data_ = [[session_id],[session_date],[item_seq]]
        #train_data = obj
        n_node = n_item+1

        model = trans_to_cuda(SessionGraph(n_node))

        best_result = [0, 0]
        best_epoch = [0, 0]
        bad_counter = 0

        for epoch in range(20):
            hit, mrr = train_test(model, train_data, test_data)
            flag = 0
            if hit > best_result[0]:
                best_result[0] = hit
                best_epoch[0] = epoch
                flag = 1
            if mrr > best_result[1]:
                best_result[1] = mrr
                best_epoch[1] = epoch
                flag = 1
            bad_counter += 1 - flag
            if bad_counter >= 5:
                break
        print('-------------------------------------------------------')
        last_sr_gnn_hit.append(hit)
        last_sr_gnn_mrr.append(mrr)

        df = make_pd_data_frame(train_data_)
        knn = cknn.ContextKNN(data = df, session_key='session_id', item_key='item_id', time_key='eventdate')

        mrr_20, p_20, scr = knn.multi_predict_ts_data(test_data, 12)


        model.eval()
        hit, mrr = [], []
        slices = test_data.generate_batch(model.batch_size)

        for i in tqdm(slices):
            targets, scores = forward(model, i, test_data)
            scores = trans_to_cpu(scores).detach().numpy()
            for j, idx in enumerate(i):
                v = scr[idx][0]
                for index, val in zip(list(v.keys()), list(v.values())):
                    scores[j,index-1] += val
            scores = torch.Tensor(scores)
            sub_scores = scores.topk(20)[1]
            sub_scores = trans_to_cpu(sub_scores).detach().numpy()
            for score, target, mask in zip(sub_scores, targets, test_data.mask):
                hit.append(np.isin(target - 1, score))
                if len(np.where(score == target - 1)[0]) == 0:
                    mrr.append(0)
                else:
                    mrr.append(1 / (np.where(score == target - 1)[0][0] + 1))

        final_hit.append(np.mean(hit)*100)
        final_mrr.append(np.mean(mrr)*100)

        sr_gnn_hit.append(best_result[0])
        sr_gnn_mrr.append(best_result[1])

        iknn_hit.append(p_20)
        iknn_mrr.append(mrr_20)
    print()
    print('SR-GNN Score: ', np.mean(last_sr_gnn_hit), np.mean(last_sr_gnn_mrr))
    print('SR-GNN Last Score: ', np.mean(sr_gnn_hit), np.mean(sr_gnn_mrr))
    print('CKNN Score: ', np.mean(iknn_hit), np.mean(iknn_mrr))
    print('Proposed Method Score: ',np.mean(final_hit), np.mean(final_mrr))
    avg_p.append(np.mean(final_hit))
    avg_m.append(np.mean(final_mrr))
print('Method 1 Avg Proposed Method Score: ',np.mean(avg_p), np.mean(avg_m))
gc.collect()

In [ ]:
for __p in range(5):
    avg_p = []
    avg_m = []
    for fold_ in range(fold):
        gc.collect()
        test_data_preproc = k_split_data_list[fold_]
        train_data_preproc = k_split_data_list.copy()
        del train_data_preproc[fold_]
        train_data_ = _ = train_data = test_data = df = knn = mrr_20 = p_20 = scr = scr_convd = slices = None

        train_data_preproc, test_data_preproc, train_data_, _ = DataFormatterA.make_tr_ts_data_set(train_data_preproc, 
                                                                                                      test_data_preproc, 
                                                                                                      sess_clicks)

        train_data = Data(train_data_preproc, shuffle=True, method=0)
        test_data = Data(test_data_preproc, shuffle=False)
        #test_data.inputs.shape
        #train_data_preproc = [[subset],[ans_items]]
        #train_data_ = [[session_id],[session_date],[item_seq]]
        #train_data = obj
        n_node = n_item+1

        model = trans_to_cuda(SessionGraph(n_node))

        best_result = [0, 0]
        best_epoch = [0, 0]
        bad_counter = 0

        for epoch in range(20):
            hit, mrr = train_test(model, train_data, test_data)
            flag = 0
            if hit > best_result[0]:
                best_result[0] = hit
                best_epoch[0] = epoch
                flag = 1
            if mrr > best_result[1]:
                best_result[1] = mrr
                best_epoch[1] = epoch
                flag = 1
            bad_counter += 1 - flag
            if bad_counter >= 5:
                break
        print('-------------------------------------------------------')
        last_sr_gnn_hit.append(hit)
        last_sr_gnn_mrr.append(mrr)

        df = make_pd_data_frame(train_data_)
        knn = cknn.ContextKNN(data = df, session_key='session_id', item_key='item_id', time_key='eventdate')

        mrr_20, p_20, scr = knn.multi_predict_ts_data(test_data, 12)


        model.eval()
        hit, mrr = [], []
        slices = test_data.generate_batch(model.batch_size)

        for i in tqdm(slices):
            targets, scores = forward(model, i, test_data)
            scores = trans_to_cpu(scores).detach().numpy()
            for j, idx in enumerate(i):
                v = scr[idx][0]
                for index, val in zip(list(v.keys()), list(v.values())):
                    scores[j,index-1] += val
            scores = torch.Tensor(scores)
            sub_scores = scores.topk(20)[1]
            sub_scores = trans_to_cpu(sub_scores).detach().numpy()
            for score, target, mask in zip(sub_scores, targets, test_data.mask):
                hit.append(np.isin(target - 1, score))
                if len(np.where(score == target - 1)[0]) == 0:
                    mrr.append(0)
                else:
                    mrr.append(1 / (np.where(score == target - 1)[0][0] + 1))

        final_hit.append(np.mean(hit)*100)
        final_mrr.append(np.mean(mrr)*100)

        sr_gnn_hit.append(best_result[0])
        sr_gnn_mrr.append(best_result[1])

        iknn_hit.append(p_20)
        iknn_mrr.append(mrr_20)
    print()
    print('SR-GNN Score: ', np.mean(last_sr_gnn_hit), np.mean(last_sr_gnn_mrr))
    print('SR-GNN Last Score: ', np.mean(sr_gnn_hit), np.mean(sr_gnn_mrr))
    print('CKNN Score: ', np.mean(iknn_hit), np.mean(iknn_mrr))
    print('Proposed Method Score: ',np.mean(final_hit), np.mean(final_mrr))
    avg_p.append(np.mean(final_hit))
    avg_m.append(np.mean(final_mrr))
print('Method 2 Avg Proposed Method Score: ',np.mean(avg_p), np.mean(avg_m))
gc.collect()

In [ ]:
for __p in range(5):
    avg_p = []
    avg_m = []
    for fold_ in range(fold):
        gc.collect()
        test_data_preproc = k_split_data_list[fold_]
        train_data_preproc = k_split_data_list.copy()
        del train_data_preproc[fold_]
        train_data_ = _ = train_data = test_data = df = knn = mrr_20 = p_20 = scr = scr_convd = slices = None

        train_data_preproc, test_data_preproc, train_data_, _ = DataFormatterA.make_tr_ts_data_set(train_data_preproc, 
                                                                                                      test_data_preproc, 
                                                                                                      sess_clicks)

        train_data = Data(train_data_preproc, shuffle=True, method=1)
        test_data = Data(test_data_preproc, shuffle=False)
        #test_data.inputs.shape
        #train_data_preproc = [[subset],[ans_items]]
        #train_data_ = [[session_id],[session_date],[item_seq]]
        #train_data = obj
        n_node = n_item+1

        model = trans_to_cuda(SessionGraph(n_node))

        best_result = [0, 0]
        best_epoch = [0, 0]
        bad_counter = 0

        for epoch in range(20):
            hit, mrr = train_test(model, train_data, test_data)
            flag = 0
            if hit > best_result[0]:
                best_result[0] = hit
                best_epoch[0] = epoch
                flag = 1
            if mrr > best_result[1]:
                best_result[1] = mrr
                best_epoch[1] = epoch
                flag = 1
            bad_counter += 1 - flag
            if bad_counter >= 5:
                break
        print('-------------------------------------------------------')
        last_sr_gnn_hit.append(hit)
        last_sr_gnn_mrr.append(mrr)

        df = make_pd_data_frame(train_data_)
        knn = cknn.ContextKNN(data = df, session_key='session_id', item_key='item_id', time_key='eventdate')

        mrr_20, p_20, scr = knn.multi_predict_ts_data(test_data, 12)


        model.eval()
        hit, mrr = [], []
        slices = test_data.generate_batch(model.batch_size)

        for i in tqdm(slices):
            targets, scores = forward(model, i, test_data)
            scores = trans_to_cpu(scores).detach().numpy()
            for j, idx in enumerate(i):
                v = scr[idx][0]
                for index, val in zip(list(v.keys()), list(v.values())):
                    scores[j,index-1] += val
            scores = torch.Tensor(scores)
            sub_scores = scores.topk(20)[1]
            sub_scores = trans_to_cpu(sub_scores).detach().numpy()
            for score, target, mask in zip(sub_scores, targets, test_data.mask):
                hit.append(np.isin(target - 1, score))
                if len(np.where(score == target - 1)[0]) == 0:
                    mrr.append(0)
                else:
                    mrr.append(1 / (np.where(score == target - 1)[0][0] + 1))

        final_hit.append(np.mean(hit)*100)
        final_mrr.append(np.mean(mrr)*100)

        sr_gnn_hit.append(best_result[0])
        sr_gnn_mrr.append(best_result[1])

        iknn_hit.append(p_20)
        iknn_mrr.append(mrr_20)
    print()
    print('SR-GNN Score: ', np.mean(last_sr_gnn_hit), np.mean(last_sr_gnn_mrr))
    print('SR-GNN Last Score: ', np.mean(sr_gnn_hit), np.mean(sr_gnn_mrr))
    print('CKNN Score: ', np.mean(iknn_hit), np.mean(iknn_mrr))
    print('Proposed Method Score: ',np.mean(final_hit), np.mean(final_mrr))
    avg_p.append(np.mean(final_hit))
    avg_m.append(np.mean(final_mrr))
print('Method 3 Avg Proposed Method Score: ',np.mean(avg_p), np.mean(avg_m))